In [1]:
#IMPORT LIBRARIES
import sqlite3
import os
import pandas as pd
import numpy as np
import random
from numpy import linalg as LA
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans, AgglomerativeClustering, MeanShift, DBSCAN, estimate_bandwidth
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from itertools import product
from math import ceil
from scipy.cluster.hierarchy import dendrogram
%matplotlib inline
#from pandas_profiling import ProfileReport 
%config InlineBackend.figure_format = 'retina' 
from scipy.stats import iqr as IQR
from collections import Counter
import scipy.stats as stat
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import coo_matrix, csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from implicit.evaluation import ranking_metrics_at_k
from tqdm import tqdm
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
import warnings
warnings.filterwarnings('ignore')

# Set seaborn style
sns.set()

In [14]:
#import file csv
df = pd.read_csv('df_pre_processed.csv', delimiter = ';')
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

In [15]:
#Show first 5 rows of data
df.head()


,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Quantity,Sales Values,Avg price
0,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,04/03/2017,2.0,1540.0,770.0
1,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,02/05/2016,4.0,3080.0,770.0
2,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,24/10/2016,2.0,1540.0,770.0
3,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,13/10/2017,2.0,1620.0,810.0
4,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,14/10/2017,2.0,1620.0,810.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ProductFamily_ID    1048575 non-null  object 
 1   ProductCategory_ID  1048575 non-null  object 
 2   ProductBrand_ID     1048575 non-null  object 
 3   ProductName_ID      1048575 non-null  object 
 4   ProductPackSKU_ID   1048575 non-null  object 
 5   Point-of-Sale_ID    1048575 non-null  object 
 6   Date                1048575 non-null  object 
 7   Quantity            1048575 non-null  float64
 8   Sales Values        1048575 non-null  float64
 9   Avg price           1048575 non-null  float64
dtypes: float64(3), object(7)
memory usage: 80.0+ MB


In [25]:
df.shape

(1048575, 10)

In [17]:
df.replace("", np.nan, inplace=True)

In [18]:
# Summary statistics for all variables
df.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ProductFamily_ID,1048575,21,Family_9,239150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ProductCategory_ID,1048575,149,Category_178,756096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ProductBrand_ID,1048575,966,ProductBrand_487,13335,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ProductName_ID,1048575,1587,ProductName_1422,12427,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ProductPackSKU_ID,1048575,4132,ProductSKU_3008,6449,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Point-of-Sale_ID,1048575,5,POS_4,384828,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Date,1048575,1401,26/04/2019,1096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,1.04858e+06,NaN,NaN,NaN,2.32259,2.27693,1,1,2,3,165
Sales Values,1.04858e+06,NaN,NaN,NaN,3786.82,7355.64,105,890,1698,3553.5,313341
Avg price,1.04858e+06,NaN,NaN,NaN,1520.54,1519.84,75,600,928,1812,17893


In [19]:
# Function to show missing values and their percentages
def missing_data(df):
    l = []
    for col in df.columns:
        missing = df[col].isnull().sum(axis=0)
        percentage = missing/df.shape[0]
        l.append([col,missing,percentage*100])
        df_missing = pd.DataFrame(l, columns = ['Feature','missing','percentage'])
        df_missing = df_missing.sort_values('missing',axis=0, ascending = False)    
    return df_missing

In [20]:
#Check Missing data
df_missing = missing_data(df)
df_missing = df_missing[df_missing['missing']>0]
df_missing

,Feature,missing,percentage


In [21]:
# Check duplicated observations
df.loc[df.duplicated(keep=False), :]

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Quantity,Sales Values,Avg price
25606,Family_1,Category_178,ProductBrand_933,ProductName_1766,ProductSKU_5344,POS_1,16/06/2016,4.0,2220.0,555.0
25608,Family_1,Category_178,ProductBrand_933,ProductName_1766,ProductSKU_5344,POS_1,18/06/2016,4.0,2220.0,555.0
25609,Family_1,Category_178,ProductBrand_933,ProductName_1766,ProductSKU_5344,POS_1,22/06/2016,2.0,1110.0,555.0
25630,Family_1,Category_178,ProductBrand_933,ProductName_1766,ProductSKU_5344,POS_1,02/07/2016,4.0,2220.0,555.0
55178,Family_12,Category_178,ProductBrand_788,ProductName_1527,ProductSKU_4668,POS_1,26/09/2018,2.0,1772.0,886.0
...,...,...,...,...,...,...,...,...,...,...
918683,Family_12,Category_178,ProductBrand_1003,ProductName_1864,ProductSKU_5656,POS_4,27/10/2018,1.0,842.0,842.0
923466,Family_9,Category_178,ProductBrand_1273,ProductName_2379,ProductSKU_7332,POS_4,29/04/2019,3.0,1377.0,459.0
923468,Family_9,Category_178,ProductBrand_1273,ProductName_2379,ProductSKU_7332,POS_4,05/02/2019,1.0,459.0,459.0
923521,Family_9,Category_178,ProductBrand_1273,ProductName_2379,ProductSKU_7332,POS_4,21/03/2019,1.0,459.0,459.0


In [22]:
# Create a dataframe copy
df1 = df.copy()

In [23]:
# Drop duplicated rows
df1.drop_duplicates(inplace=True)

In [24]:
print('Percentage of data kept after removing irregularities:', np.round(df1.shape[0] / df.shape[0], 4))

Percentage of data kept after removing irregularities: 0.9999


In [30]:
# Define non-metric features
non_metric_features = ['ProductFamily_ID','ProductCategory_ID','ProductBrand_ID','ProductName_ID','ProductPackSKU_ID','Point-of-Sale_ID','Date']
metric_features = ['Quantity','Sales Values','Avg price']

In [29]:
df = pd.read_excel('df_demand.xlsx', delimiter = ';')
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

FileNotFoundError: [Errno 2] No such file or directory: 'df_demand.xlsx'